In [0]:
import pandas as pd

from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RandomizedSearchCV


In [0]:
!gdown --id 1Q32n6md9vtxL02teZ1TD6ZMWdKkcSt89

Downloading...
From: https://drive.google.com/uc?id=1Q32n6md9vtxL02teZ1TD6ZMWdKkcSt89
To: /content/aggregatedData.csv
727MB [00:03, 220MB/s]


In [0]:
df = pd.read_csv('aggregatedData.csv')

In [0]:
df

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
0,141227,-1893.0,38.03651,112.0,49.0,158.00000,1.40,48.20,4.300000,2.433090,0
1,141227,-1773.0,38.03651,112.0,49.0,158.00000,1.40,48.20,4.300000,2.433090,0
2,141227,-1663.0,38.03651,112.0,49.0,158.00000,1.40,48.20,4.300000,2.433090,0
3,141227,-1566.0,38.03651,112.0,49.0,158.00000,1.40,48.20,4.300000,2.433090,0
4,141227,-1351.0,38.03651,112.0,49.0,158.00000,1.40,47.95,4.300000,2.433090,0
...,...,...,...,...,...,...,...,...,...,...,...
6042126,3353254,5326.0,38.03651,82.0,12.0,118.56613,1.65,11.32,2.655717,4.767580,0
6042127,3353254,5491.0,38.03651,82.0,12.0,118.56613,1.66,11.51,2.655717,2.383790,0
6042128,3353254,5558.0,38.03651,82.0,12.0,118.56613,1.67,11.70,2.655717,4.767580,0
6042129,3353254,5926.0,38.03651,82.0,12.0,118.56613,1.67,11.70,2.655717,7.151371,0


In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [0]:
target = df['diagnosis']
inputs = df.drop('diagnosis', axis=1).copy()

In [0]:
X = inputs.values
y = target.values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [0]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4531598, 10), (4531598,), (1510533, 10), (1510533,))

In [0]:
import numpy as np
X_train = torch.FloatTensor(np.array(X_train))
X_test = torch.FloatTensor(np.array(X_test))
y_train = torch.FloatTensor(np.array(y_train))
y_test = torch.FloatTensor(np.array(y_test))

In [0]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output

In [0]:
model = Feedforward(10, 10)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
model.to(device)

Feedforward(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [0]:
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

In [0]:
model.eval()
y_pred = model(X_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

Test loss before training 0.2720893919467926


In [0]:
model.train()
epoch = 200
for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(X_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.27255284786224365
Epoch 1: train loss: 0.27255284786224365
Epoch 2: train loss: 0.27255284786224365
Epoch 3: train loss: 0.27255284786224365
Epoch 4: train loss: 0.27255284786224365
Epoch 5: train loss: 0.27255284786224365
Epoch 6: train loss: 0.27255284786224365
Epoch 7: train loss: 0.27255284786224365
Epoch 8: train loss: 0.27255284786224365
Epoch 9: train loss: 0.27255284786224365
Epoch 10: train loss: 0.27255284786224365
Epoch 11: train loss: 0.27255284786224365
Epoch 12: train loss: 0.27255284786224365
Epoch 13: train loss: 0.27255284786224365
Epoch 14: train loss: 0.27255284786224365
Epoch 15: train loss: 0.27255284786224365
Epoch 16: train loss: 0.27255284786224365
Epoch 17: train loss: 0.27255284786224365
Epoch 18: train loss: 0.27255284786224365
Epoch 19: train loss: 0.27255284786224365
Epoch 20: train loss: 0.27255284786224365
Epoch 21: train loss: 0.27255284786224365
Epoch 22: train loss: 0.27255284786224365
Epoch 23: train loss: 0.27255284786224365
Ep

In [0]:
model.eval()
y_pred = model(X_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 0.2720893919467926
